In [1]:
from affine import *
from common import *
from conditions import *
from cycle import *

# 最もナイーブな方法(ランダムに生成して、条件を満たすものが見つかるまで探索し続ける)

In [10]:
# count = 0
# while(1):
#     count += 1
#     if count % 1000 == 0:
#         print("count: ", count)
#     f_list = [generate_random_affine_function()]*l_h
#     g_list = [generate_random_affine_function()]*l_h
#     if check_conditions(f_list, g_list):
#         print(f_list, g_list)
#         break

# 先に$f$のリストを作り、条件(a)の解空間に含まれるものをランダムに生成し、条件(b),(c)を確かめる

In [2]:
cols_list = make_cols_list()

In [3]:
check_c_results = []
check_c_count = 0
for i in range(10000):
    flag_f = False
    flag_g = False
    not_found = False
    count_f = 0
    count_g = 0
    f_list = []
    while(flag_f is False and not_found is False):
        if count_f == 50:
            not_found = True
        for _ in range(l_h):
            f_list.append(generate_random_affine_function())
        flag_f = check_inner_commute(f_list)
        count_f += 1
    g_candidates = []
    for i in range(l_h):
        g_candidates.append(commute_with(f_list[i]))
    g_candidates_list = list(set(commute_with(f_list[0]))&set(commute_with(f_list[1]))&set(commute_with(f_list[2])))
    while(flag_g is False and not_found is False):
        if count_g == 50:
            not_found = True
        g_list = []
        for _ in range(l_h):
            g_list.append(random.choice(g_candidates_list))
        flag_g = check_inner_commute(g_list)
        count_g += 1
    if not_found:
        continue
    # print(f_list, g_list)
    result = check_condition_c(f_list, g_list, cols_list)
    if result is True:
        print("check success!")
        print(f_list)
        print(g_list)
        break
    else:
        check_c_count += 1
        check_c_results.extend(result)

counts = Counter(check_c_results)
print(check_c_count, len(check_c_results), len(check_c_results)/check_c_count)
print(counts)

50 140600 2812.0
Counter({(0, 1): 50, (2, 4): 50, (1, 2): 50, (0, 4): 50, (3, 4): 50, (1, 5): 50, (0, 3): 50, (1, 4): 50, (2, 3): 50, (0, 2): 50, (4, 5): 50, (0, 5): 50, (2, 5): 50, (1, 3): 50, (3, 5): 50, (0, 1, 4, 5): 50, (0, 4, 0, 4): 50, (0, 1, 5, 4): 50, (1, 3, 5, 2): 50, (0, 5, 2, 4): 50, (0, 5, 4, 1): 50, (1, 3, 2, 3): 50, (1, 5, 2, 5): 50, (1, 5, 4, 2): 50, (0, 4, 1, 5): 50, (0, 4, 2, 4): 50, (1, 4, 2, 3): 50, (0, 3, 1, 3): 50, (0, 5, 1, 5): 50, (0, 1, 2, 1): 50, (0, 5, 3, 2): 50, (1, 3, 1, 4): 50, (1, 3, 5, 4): 50, (0, 3, 2, 4): 50, (1, 2, 1, 3): 50, (2, 5, 3, 5): 50, (1, 3, 2, 5): 50, (1, 4, 2, 5): 50, (0, 3, 1, 5): 50, (2, 3, 5, 3): 50, (0, 1, 2, 3): 50, (0, 5, 3, 4): 50, (0, 1, 3, 2): 50, (1, 5, 3, 2): 50, (1, 2, 1, 5): 50, (1, 2, 3, 2): 50, (0, 3, 5, 2): 50, (0, 2, 1, 2): 50, (0, 2, 4, 1): 50, (1, 2, 5, 2): 50, (0, 1, 2, 5): 50, (0, 1, 4, 2): 50, (0, 1, 3, 4): 50, (0, 1, 5, 1): 50, (0, 2, 3, 2): 50, (1, 5, 3, 4): 50, (3, 4, 3, 4): 50, (0, 5, 2, 1): 50, (0, 2, 0, 3): 50, (1

In [5]:
print(check_c_count, len(check_c_results), len(check_c_results)/check_c_count)

57 160284 2812.0


出力: 57 160284 2812.0

# 行列演算による条件a,bを満たす解空間の計算

In [1]:
from affine import *
from common import *
from conditions import *
from cycle import *

In [ ]:
# f_a_vec = np.array([2,3,4])
# g_a_vec = np.array([5,6,7])
# a_vec = np.concatenate([f_a_vec, g_a_vec])
# f_b_vec = np.array([])
# g_b_vec = np.array([])
# b_vec = np.concatenate([f_b_vec, g_b_vec])

In [2]:
def generate_random_a_vec():
    a_vec = []
    prime_list = []
    for i in range(2,P):
        if math.gcd(i, P) == 1:
            prime_list.append(i)
    for _ in range(L):
        val = random.choice(prime_list)
        a_vec.append(val)
    return np.array(a_vec)

In [3]:
a_vec = generate_random_a_vec()
a_vec

array([233, 211,  11,  89,   7, 367])

In [4]:
cond_a_mat = np.array([
    [1-a_vec[3], 0, 0, a_vec[0]-1, 0, 0],
    [1-a_vec[4], 0, 0, 0, a_vec[0]-1, 0],
    [1-a_vec[5], 0, 0, 0, 0, a_vec[0]-1],
    [0, 1-a_vec[3], 0, a_vec[1]-1, 0, 0],
    [0, 1-a_vec[4], 0, 0, a_vec[1]-1, 0],
    [0, 1-a_vec[5], 0, 0, 0, a_vec[1]-1],
    [0, 0, 1-a_vec[3], a_vec[2]-1, 0, 0],
    [0, 0, 1-a_vec[4], 0, a_vec[2]-1, 0],
    [0, 0, 1-a_vec[5], 0, 0, a_vec[2]-1],
])
cond_b_f_mat = np.array([
    [1-a_vec[1], a_vec[0]-1, 0, 0, 0, 0],
    [1-a_vec[2], 0, a_vec[0]-1, 0, 0, 0],
    [0, 1-a_vec[2], a_vec[1]-1, 0, 0, 0],
])
cond_b_g_mat = np.array([
    [0, 0, 0, 1-a_vec[4], a_vec[3]-1, 0],
    [0, 0, 0, 1-a_vec[5], 0, a_vec[3]-1],
    [0, 0, 0, 0, 1-a_vec[5], a_vec[4]-1],
])

In [ ]:
from sympy import Matrix, mod_inverse

def mod_rref(matrix):
    mat = [row[:] for row in matrix]
    rows = len(mat)
    cols = len(mat[0])
    pivot_row = 0
    pivot_cols = []
    
    for j in range(cols):
        if pivot_row >= rows:
            break
        
        # ピボットの選択
        sel = -1
        for i in range(pivot_row, rows):
            if mat[i][j] % P != 0:
                sel = i
                break
        
        if sel == -1:
            continue
            
        # 行の入れ替え
        mat[pivot_row], mat[sel] = mat[sel], mat[pivot_row]
        
        # ピボット行の正規化（逆元を掛ける）
        inv = mod_inverse(mat[pivot_row][j] % P, P)
        for k in range(j, cols):
            mat[pivot_row][k] = (mat[pivot_row][k] * inv) % P
            
        # 他の行の掃き出し
        for i in range(rows):
            if i != pivot_row:
                factor = mat[i][j] % P
                for k in range(j, cols):
                    mat[i][k] = (mat[i][k] - factor * mat[pivot_row][k]) % P
        
        pivot_cols.append(j)
        pivot_row += 1
        
    return mat, pivot_cols

In [5]:
cond_a_mat

array([[ -88,    0,    0,  232,    0,    0],
       [  -6,    0,    0,    0,  232,    0],
       [-366,    0,    0,    0,    0,  232],
       [   0,  -88,    0,  210,    0,    0],
       [   0,   -6,    0,    0,  210,    0],
       [   0, -366,    0,    0,    0,  210],
       [   0,    0,  -88,   10,    0,    0],
       [   0,    0,   -6,    0,   10,    0],
       [   0,    0, -366,    0,    0,   10]])